In [2]:
# 根据 CoM 坐标和足的坐标计算蹬伸角度

In [16]:
import pandas as pd
import math
import os

# 读取 Excel 文件
def read_excel(file_path):
    com_df = pd.read_excel(file_path, sheet_name='Center of Mass')
    seg_pos_df = pd.read_excel(file_path, sheet_name='Segment Position')
    return com_df, seg_pos_df

# 计算蹬伸向量的角度和半径
def calculate_vector_properties(com_y, com_z, foot_y, foot_z):
    # 计算向量
    vector_y = com_y - foot_y
    vector_z = com_z - foot_z
    length = math.sqrt(vector_y**2 + vector_z**2)
    length_round = round(length,2)
    if length != 0:  # 防止除以零
        angle_radians = math.acos(vector_y / length)
        # 转换为角度
        angle_degrees = math.degrees(angle_radians)
    else:
        angle_degrees = 0

    angle_degrees_round = round(angle_degrees,2)
    return length_round, angle_degrees_round

# 主处理函数
def process_file(file_path):
    com_df, seg_pos_df = read_excel(file_path)

    # 创建结果列表
    results = []

    # 对每一行数据进行处理
    for index, row in seg_pos_df.iterrows():
        if index < len(com_df):
            # 计算左脚的蹬伸角和半径
            left_length, left_angle = calculate_vector_properties(
                com_df.at[index, 'CoM pos y'], com_df.at[index, 'CoM pos z'],
                row['Left Foot y'], row['Left Foot z']
            )
            
            # 计算右脚的蹬伸角和半径
            right_length, right_angle = calculate_vector_properties(
                com_df.at[index, 'CoM pos y'], com_df.at[index, 'CoM pos z'],
                row['Right Foot y'], row['Right Foot z']
            )

            # 添加结果到列表
            results.append({'Frame': row['Frame'], 'Left Slip Angle': left_angle, 'Left Slip Radius': left_length, 
                            'Right Slip Angle': right_angle, 'Right Slip Radius': right_length})

    # 创建新的 DataFrame
    results_df = pd.DataFrame(results)

    # 保存结果到新的 Excel 文件
    new_file_path = os.path.splitext(file_path)[0] + '_蹬伸角_半径.xlsx'
    results_df.to_excel(new_file_path, index=False)


In [ ]:
# 替换为您的 Excel 文件路径
file_path = '/Users/wangshuaibo/Downloads/20231019_1_lininghao-001.xlsx'
process_file(file_path)